# Centrality Notebook

In this notebook we use NetworKit to compute popular centrality measures.

In [ ]:
import networkit as nk
from networkit import csbridge
import seaborn as sns

Read a graph

In [ ]:
g = nk.graphio.KONECTGraphReader().read('../input/karate')
print(f'The graph has {g.numberOfNodes()} nodes and {g.numberOfEdges()} edges.')

# Algebraic Centrality Measures

### Eigenvector Centrality

$$
c(u) = \frac{1}{\lambda}\sum_{v\in N(u)}c(v)
$$

In [ ]:
# 1. Initialize algorithm
ec = nk.centrality.EigenvectorCentrality(g, tol=1e-9)

# 2. Run the algorithm
ec.run()

# 3. Retrieve the results
ec_ranking = ec.ranking()

# Print the top-5 nodes
print("Node\tScore")
for u, score in ec_ranking[:5]:
    print("{:d}\t{:.2f}".format(u, score))

In [ ]:
csbridge.widget_from_graph(g, node_scores=ec.scores(), node_palette=sns.color_palette("vlag"))

## PageRank

$$
PR(u) = \frac{1 - d}{n} + d\sum_{v \in N(u)}\frac{PR(v)}{\deg(v)}
$$

In [ ]:
# 1. Initialize algorithm
pr = nk.centrality.PageRank(g, damp=0.85, tol=1e-9)

# Choose the norm used for convergence (default: l2 norm)
pr.norm = nk.centrality.Norm.l1norm

# 2. Run the algorithm
pr.run()

# 3. Retrieve the results
pr_ranking = pr.ranking()

# Print top-5 nodes
print("Node\tScore")
for u, score in pr_ranking[:5]:
    print("{:d}\t{:.2f}".format(u, score))

In [ ]:
csbridge.widget_from_graph(g, node_scores=pr.scores(), node_palette=sns.color_palette("vlag"))

# Shortest-path-based Centrality Measures

## Betweenness Centrality

$\delta_{st}$: number of $s$-$t$ shortest paths.

$\delta_{st}(u)$: fraction of $s$-$t$ shortest paths crossing $u$.

$$
bc(u) = \sum_{s \neq u \neq v \in V}\frac{\delta_{st}(u)}{\delta_{st}}
$$

Using Brandes's algorithm:

In [ ]:
bc = nk.centrality.Betweenness(g, normalized=True, computeEdgeCentrality=False)

bc.run()

bc_ranking = bc.ranking()

# Print top-5 nodes
print("Node\tScore")
for u, score in bc_ranking[:5]:
    print("{:d}\t{:.2f}".format(u, score))

In [ ]:
csbridge.widget_from_graph(g, node_scores=bc.scores(), node_palette=sns.color_palette("vlag"))

## Betweenness Approximation
Exact computation unfeasible in large graphs.

Kadabra algorithm: $\varepsilon$-approximation with high probability (or top-$k$ ranking) with parallel adaptive sampling.

In [ ]:
g_large = nk.graphio.METISGraphReader().read('../input/PGPgiantcompo.graph')
print("The graph has {:,} nodes and {:,} edges.".format(g_large.numberOfNodes(), g_large.numberOfEdges()))

Run the Kadabra algorithm:

In [ ]:
# eps-approx for all the vertices in the graph
kb = nk.centrality.KadabraBetweenness(g_large, err=0.01).run()
# exact top-k computation
#kb = nk.centrality.KadabraBetweenness(g_large, k=5).run()

bc_ranking = kb.ranking()

print("Node\tScore")
for u, score in bc_ranking[:5]:
    print("{:d}\t{:.3f}".format(u, score))

## Harmonic Centrality

$$
hc(u) = \frac{1}{n-1}\sum_{v \neq u}\frac{1}{d(u, v)}
$$

In [ ]:
hc = nk.centrality.HarmonicCloseness(g, normalized=True).run()
hc_ranking = hc.ranking()

# Print top-5 nodes
print("Node\tScore")
for u, score in hc_ranking[:5]:
    print("{:d}\t{:.2f}".format(u, score))

In [ ]:
csbridge.widget_from_graph(g, node_scores=hc.scores(), node_palette=sns.color_palette("vlag"))

## Top-$k$ Harmonic Centrality
Naive algorithm: BFS/Dijkstra from each vertex $\leadsto$ unfeasible on large graphs.

Often, only the top-$k$ most important vertices are required.

Efficient and parallel top-$k$ harmonic centrality computation (Bergamini et al., ALENEX 2016).

In [ ]:
top_hc = nk.centrality.TopHarmonicCloseness(g_large, k=5, useNBbound=False).run()

# Print top-5 nodes
print("Node\tScore")
for u, score in zip(top_hc.topkNodesList(), top_hc.topkScoresList()):
    print("{:d}\t{:.2f}".format(u, score))

# Exercise 1

Do you think your number of online friends is above/below/on average? (You do not have to answer this question openly.) Answer (may be secret):

What do you expect: How many people (in percent) in a social network have fewer friends than their friends on average? Answer (choose one): a) 0 - 25% b) 26 - 50% c) 51 - 75% d) 76 - 100%

Use the Facebook graph. Compute for each vertex the average degree of its neighbors. Answer:

Count the number of persons whose friends have on average more friends. What is their percentage in this network? Answer:

In [ ]:
fb = nk.graphio.SNAPGraphReader().read('../input/facebook')
print('This graph has {:,} nodes and {:,} edges'.format(fb.numberOfNodes(), fb.numberOfEdges()))

In [ ]:
# Write your solution here
# Use:
# - fb.iterNodes() to iterate over all the nodes of a graph
# - fb.iterNeighbors(u) to iterate over the neighbors of u
# - fb.degree(u) to get the degree of u

# Exercise 2

A Song of Ice and Fire is a very long novel with many characters.

Here we will try to identify important characters according to how much they interact with other characters.

The graph that we use is taken from https://github.com/mathbeveridge/asoiaf, every vertex is a character and every edge represents interactions between the characters.

In [ ]:
# Read the graph
got = nk.graphio.EdgeListReader(separator=' ', firstNode=0, continuous=True, directed=False).read('../input/got_edges.txt')
print(f'There are {got.numberOfNodes()} characters and {got.numberOfEdges()} interactions.')

# Read the character list
characters = []
with open('../input/got_names.txt', 'r') as f:
    for character in f.readlines():
        characters.append(character.strip())

idx = 48
print(f'The character at index {idx} is {characters[idx]}')

### $k$-core decomposition to identify the main characters

$k$-core: maximal connected subgraph in which all vertices have degree $ \ge k$.

Use the `CoreDecomposition` algorithm (`community` module) in NetworKit to identify the characters with the most inner core.

The core in which every character belongs can be extracted using the `scores()` function in `CoreDecomposition`.

The most inner core is the one with highest score. The highest core number can be obtained from the `maxCoreNumber()` function in `CoreDecomposition`.

In [ ]:
# Write your solution here

### Most central character
Which character is the most important according to the centrality measures we experimented above? Is this expected?

In [ ]:
centr = nk.centrality.Betweenness(got).run()
print(characters[centr.ranking()[0][0]])